In [1]:
# https://vimsky.com/zh-tw/examples/detail/python-method-pymongo.MongoClient.html
# https://vimsky.com/zh-tw/examples/detail/python-method-pymongo.collection.html

from pymongo import MongoClient
from bson.tz_util import FixedOffset
from bson.objectid import ObjectId
# from datetime import datetime
import os
import pandas as pd
import time
import datetime
from time import mktime as mktime

In [2]:
'''
os.getenv("admin", "mongodb://name:XXX:27017")
'''

myclient = MongoClient(os.getenv("admin", "mongodb://name:XXX:27017"))
# myclient.database_names()
mydb = myclient["insula_staging"]
# mydb.collection_names()
collection = mydb["game_analysis"]

In [3]:
collection.count()

<ipython-input-3-7c4bc5758cb0>:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  collection.count()


923

In [4]:
def trans_time(time1):
#     t1 = datetime.datetime.strptime(time, "%Y-%m-%d %H:%M:%S.%f")
    t2 = time1 + datetime.timedelta(hours = 8)
    #轉成時間戳
    #10秒
    date_stamp = str(int(mktime(t2.timetuple())))
    #3秒
    data_microsecond = str("%06d"%t2.microsecond)[0:3]
    date_stamp = date_stamp+data_microsecond
    
    return date_stamp

In [5]:
def column_order(df,feature_list):
    feature_list.remove('email')
    
    cols = df.columns.to_list()
    cols.insert(0, cols.pop(cols.index('userid')))
    
    for i in feature_list:
        cols.insert(cols.index('userid')+1, cols.pop(cols.index(i)))
    
    df_f = df[cols]
    
    return df_f

In [6]:
'''
Data
'''
def mongo_concat_data(cursor, feature_list):
    df_final = pd.DataFrame()
    for i in range(cursor.count()):
        df_1 = pd.DataFrame(cursor[i]['data'])
        for j in feature_list:
            df_1.loc[:,j] = cursor[i].get(j)
        
        df_1.loc[:,'start_time'] = df_1.loc[:,'start_time'].apply(lambda x : trans_time(x))
        df_1.loc[:,'end_time'] = df_1.loc[:,'end_time'].apply(lambda x : trans_time(x))
        
        #MG5
        try:
            for k in cursor[i]['score'].keys():
                df_1.loc[:,k] = cursor[i]['score'].get(k)
        except:
            pass                
            
        df_final = pd.concat([df_final, df_1], axis= 0)
        
    # 增加與調整userid
    df_final.loc[:,'userid'] = df_final.loc[:,'email'].apply(lambda x: x.split('@')[0])
    df_final.drop(["email"], axis=1,inplace = True)
    
    #調整順序
    df_final = column_order(df_final,feature_list)
    
    df_final = df_final.reset_index(drop = True)
    return df_final    

In [7]:
'''
Event
'''
def mongo_concat_event(cursor): 
    
    df_final = pd.DataFrame()
    for i in range(cursor.count()):
        df_1 = pd.DataFrame(cursor[i]['event'])
        df_1.loc[:,'game'] = cursor[i].get('game')
        df_1.loc[:,'userid'] = cursor[i].get('email')
        df_1.loc[:,'userid'] = df_1.loc[:,'userid'].apply(lambda x: x.split('@')[0])
        df_1.loc[:,'time'] = df_1.loc[:,'time'].apply(lambda x : trans_time(x))
        df_final = pd.concat([df_final, df_1], axis= 0)
        
    #調整順序
    cols = df_final.columns.to_list()
    cols.insert(0, cols.pop(cols.index('userid')))
    cols.insert(cols.index('userid')+1, cols.pop(cols.index('game')))
    df_final = df_final[cols]        
    df_final = df_final.reset_index(drop = True)
    
#     df_final.loc[:,'time'] = df_final.loc[:,'time'].apply(lambda x : trans_time(x))

    return df_final    

In [8]:
def df_to_csv(df_data, df_event, mg_path, csv_name):
    
    try:
        df_data.to_csv(mg_path +'/'+csv_name+'all_data.csv', index = False, encoding= "utf_8_sig")
        print(csv_name + '_data轉檔成功')
    except:
        print(csv_name + '_data轉檔失敗')
    
    try:
        df_event.to_csv(mg_path +'/'+csv_name+'all_event.csv', index = False, encoding= "utf_8_sig")
        print(csv_name + '_event轉檔成功')
    except:
        print(csv_name + '_event轉檔失敗')        

In [ ]:
'''

https://stackoverflow.com/questions/3305561/how-to-query-mongodb-with-like
dict_keys(['_id', 'version', 'created_at', 'user_id', 'email', 'game', 'stage', 'start_time', 'end_time', 'mngs', 'data', 'event'])
"start_time" : { "$gte" : datetime.strptime('2021-08-12 08:00:00.000000', "%Y-%m-%d %H:%M:%S.%f").replace(tzinfo = FixedOffset(480, "+0800"))}
'email' : { "$in" : ['']}

"start_time" : { "$gte" : datetime.datetime.strptime('2021-09-24 08:00:00.000000', "%Y-%m-%d %H:%M:%S.%f").replace(tzinfo = FixedOffset(480, "+0800")),
"$lte" : datetime.datetime.strptime('2021-09-25 08:00:00.000000', "%Y-%m-%d %H:%M:%S.%f").replace(tzinfo = FixedOffset(480, "+0800"))}
'email': {'$regex': 'YP'}
"created_at" : {"$gte" : datetime.datetime(2021, 9, 24), "$lte" : datetime.datetime(2021, 9, 25)
"created_at" : {"$gte" : datetime.datetime(2021, 10, 1), "$lte" : datetime.datetime(2021, 10, 2)}
'''
cursor = collection.find({'game': 'C',
                          
                          'email': {'$regex': 'XX', '$options': 'XX'}
                          
                                                    
                         })


mg_path = r'XXX'
feature_list = ['email', 'game', 'mngs', 'start_time', 'end_time']
csv_name = 'XXX'
df_data = mongo_concat_data(cursor,feature_list)
df_event = mongo_concat_event(cursor)
df_to_csv(df_data, df_event, mg_path, csv_name)
display(df_data.userid.unique())
display(df_data.head())